# Using `PacMan`

This notebook contains examples on how to use the class `PacMan`.

In [ ]:
from pytket import Circuit
from pytket.circuit import OpType, Op
from pytket.circuit.display import render_circuit_jupyter
from pytket_dqc.networks import NISQNetwork
from pytket_dqc.allocators import HypergraphPartitioning
from pytket_dqc.circuits import HypergraphCircuit
from pytket_dqc.packing import PacMan
from pytket_dqc.utils import DQCPass
from pytket_dqc.placement import Placement
from sympy import Symbol
import networkx as nx
import matplotlib.pyplot as plt

cz = Op.create(OpType.CU1, 1)

`PacMan` can find all sorts of `CU1` gates that can be distributed together.  
For the purposes of these examples, we imagine that the circuit is distributed across 3 servers, each with 2 qubits.  
The qubits 0 & 1 are on server 0, qubits 2 & 3 on server 1, qubits 4 & 5 on server 2.  
The `Placements` are manually constructed to illustrate this desired behaviour.

The simplest examples are `CU1` gates separated by single qubit diagonal gates.

In [ ]:
# Custom placement
placement_dict00 = dict()
for i in range(6):
    placement_dict00[i] = i // 2
placement_dict00[6] = 0
placement_dict00[7] = 0
placement00 = Placement(placement_dict00)

# Build circuit
circ00 = Circuit(6)
circ00.add_gate(cz, [0,2])
circ00.Rz(0.3, 0)
circ00.add_gate(cz, [0,3])
render_circuit_jupyter(circ00)

# Make hypergraph of circuit
h_circ00 = HypergraphCircuit(circ00)

# Pass into PacMan
pacman00 = PacMan(h_circ00, placement00)

`CU1` gates separated by single qubit anti-diagonal gates can also be distributed together.

In [ ]:
# Custom placement
placement_dict01 = dict()
for i in range(6):
    placement_dict01[i] = i // 2
placement_dict01[6] = 0
placement_dict01[7] = 0
placement01 = Placement(placement_dict01)

# Build circuit
circ01 = Circuit(6)
circ01.add_gate(cz, [0,2])
circ01.Rz(0.3, 0)
# Note H Rz(1) H = H Z H = X, an anti-diagonal gate.
circ01.H(0)
circ01.Rz(1, 0)
circ01.H(0)
circ01.add_gate(cz, [0,3])
render_circuit_jupyter(circ01)

# Make hypergraph of circuit
h_circ01 = HypergraphCircuit(circ01)

# Pass into PacMan
pacman01 = PacMan(h_circ01, placement01)

`CU1` gates that go to different servers are split into different packets.

In [ ]:
# Custom placement
placement_dict02 = dict()
for i in range(6):
    placement_dict02[i] = i // 2
placement_dict02[6] = 0
placement_dict02[7] = 0
placement_dict02[8] = 0
placement_dict02[9] = 0
placement02 = Placement(placement_dict02)

# Build circuit
circ02 = Circuit(6)
circ02.add_gate(cz, [0,2])
circ02.add_gate(cz, [0,4])
circ02.add_gate(cz, [0,3])
circ02.add_gate(cz, [0,5])
render_circuit_jupyter(circ02)

# Make hypergraph of circuit
h_circ02 = HypergraphCircuit(circ02)

# Pass into PacMan
pacman02 = PacMan(h_circ02, placement02)

The above 3 examples demonstrate neighbouring packing.  
Provided the right conditions are met, we can also merge packets via hopping packing.

The simplest case of this is two Hadamards surrounding a CZ gate (= CX gate).

In [ ]:
# Custom placement
placement_dict03 = dict()
for i in range(6):
    placement_dict03[i] = i // 2
placement_dict03[6] = 0
placement_dict03[7] = 0
placement_dict03[8] = 0
placement03 = Placement(placement_dict03)

# Build circuit
circ03 = Circuit(6)
circ03.add_gate(cz, [0,2])
circ03.H(0)
circ03.add_gate(cz, [0,3])
circ03.H(0)
circ03.add_gate(cz, [0,2])
render_circuit_jupyter(circ03)

# Make hypergraph of circuit
h_circ03 = HypergraphCircuit(circ03)

# Pass into PacMan
pacman03 = PacMan(h_circ03, placement03)

print(pacman03.hopping_packets)

More complicated hopping packings can also be identified.

Two embedded `CU1`s with a single `Z` between them.

In [ ]:
# Custom placement
placement_dict04 = dict()
for i in range(6):
    placement_dict04[i] = i // 2
placement_dict04[6] = 0
placement_dict04[7] = 0
placement_dict04[8] = 0
placement04 = Placement(placement_dict04)

# Build circuit
circ04 = Circuit(6)
circ04.add_gate(cz, [0,2])
circ04.H(0)
circ04.add_gate(cz, [0,2])
circ04.Rz(1, 0)
circ04.add_gate(cz, [0,3])
circ04.H(0)
circ04.add_gate(cz, [0,2])
render_circuit_jupyter(circ04)

# Make hypergraph of circuit
h_circ04 = HypergraphCircuit(circ04)

# Pass into PacMan
pacman04 = PacMan(h_circ04, placement04)

print(pacman04.hopping_packets)

Or with an `X` between them.

In [ ]:
# Custom placement
placement_dict05 = dict()
for i in range(6):
    placement_dict05[i] = i // 2
placement_dict05[6] = 0
placement_dict05[7] = 0
placement_dict05[8] = 0
placement_dict05[9] = 0
placement05 = Placement(placement_dict05)

# Build circuit
circ05 = Circuit(6)
circ05.add_gate(cz, [0,2])
circ05.H(0)
circ05.add_gate(cz, [0,2])
# Note H Rz(1) H = H Z H = X, an anti-diagonal gate.
circ05.H(0)
circ05.Rz(1, 0)
circ05.H(0)
circ05.add_gate(cz, [0,3])
circ05.H(0)
circ05.add_gate(cz, [0,2])
render_circuit_jupyter(circ05)

# Make hypergraph of circuit
h_circ05 = HypergraphCircuit(circ05)

# Pass into PacMan
pacman05 = PacMan(h_circ05, placement05)

print(pacman05.hopping_packets)

(Actually in this case the `Rz` can have any phase)

In [ ]:
# Custom placement
placement_dict06 = dict()
for i in range(6):
    placement_dict06[i] = i // 2
placement_dict06[6] = 0
placement_dict06[7] = 0
placement_dict06[8] = 0
placement06 = Placement(placement_dict06)

# Build circuit
circ06 = Circuit(6)
circ06.add_gate(cz, [0,2])
circ06.H(0)
circ06.add_gate(cz, [0,2])
circ06.H(0)
circ06.Rz(0.27, 0)
circ06.H(0)
circ06.add_gate(cz, [0,3])
circ06.H(0)
circ06.add_gate(cz, [0,2])
render_circuit_jupyter(circ06)

# Make hypergraph of circuit
h_circ06 = HypergraphCircuit(circ06)

# Pass into PacMan
pacman06 = PacMan(h_circ06, placement06)

print(pacman06.hopping_packets)

There may just be 1 Hadamard between two embedded `CU1`s.

In [ ]:
# Custom placement
placement_dict07 = dict()
for i in range(6):
    placement_dict07[i] = i // 2
placement_dict07[6] = 0
placement_dict07[7] = 0
placement_dict07[8] = 0
placement07 = Placement(placement_dict07)

# Build circuit
circ07 = Circuit(6)
circ07.add_gate(cz, [0,2])
circ07.H(0)
circ07.add_gate(cz, [0,2])
circ07.Rz(0.5, 0)
circ07.H(0)
circ07.Rz(0.5, 0)
circ07.add_gate(cz, [0,3])
circ07.H(0)
circ07.add_gate(cz, [0,2])
render_circuit_jupyter(circ07)

# Make hypergraph of circuit
h_circ07 = HypergraphCircuit(circ07)

# Pass into PacMan
pacman07 = PacMan(h_circ07, placement07)

print(pacman07.hopping_packets)

Graph methods are useful for analysing the ebit cost of these packings.  
We can use the method `.get_nx_graph_merged()` to return a graph representation of the circuit.

Note that this method also returns all the nodes on one half of the bipartite graph (conventionally called the top half). This is because the graph is likely highly disconnected, so `NetworkX` requires this to avoid ambiguous solutions to certain methods.

In [ ]:
g, topnodes = pacman.get_nx_graph_merged()
print(nx.adjacency_matrix(g).toarray())

This can be easily drawn as a bipartite graph like so (note this is an example of needing `topnodes`).

In [ ]:
nx.draw_networkx(
    g,
    pos = nx.drawing.layout.bipartite_layout(g, topnodes), 
    width = 1)

We can also get and draw the conflict graph.

In [ ]:
conflict_g, conflict_topnodes = pacman.get_nx_graph_conflict()
nx.draw_networkx(
    conflict_g,
    pos = nx.drawing.layout.bipartite_layout(conflict_g, conflict_topnodes), 
    width = 1)

For our purposes, it is useful to obtain a minimum vertex cover of the merged graph, as well as find out which vertices of the MVC also form conflicting edges (here, the conflicting edge does not represent a true conflict).

In [ ]:
print(pacman.get_mvc_merged_graph())

## More complex circuit (under construction)

We now import a Chemistry circuit as a real world example.

In [ ]:
import json
# Replace this with the location of the Chemistry circuit.
with open("../../Tokyo-CQC-collab/circuits/uccsd/raw/06_02_01.json") as f:
    circuit = Circuit.from_dict(json.load(f))
network = NISQNetwork(
    [[0,1]],
    {0: [0,1,2], 1: [3,4,5]}
)

# Bind all the symbols in the circuit
t01 = Symbol("t0_1")
t02 = Symbol("t0_2")
t11 = Symbol("t1_1")
t12 = Symbol("t1_2")
t22 = Symbol("t2_2")
t21 = Symbol("t2_1")
t31 = Symbol("t3_1")
t41 = Symbol("t4_1")
t51 = Symbol("t5_1")
t61 = Symbol("t6_1")
t71 = Symbol("t7_1")

symbols = [
    t01, t02, t11, t12, t22, t21, t31, t41, t51, t61, t71
]

symbol_map = dict()

for symbol in symbols:
    symbol_map[symbol] = 0

circuit.symbol_substitution(symbol_map)
render_circuit_jupyter(circuit)

# Check they're all bound
assert len(circuit.free_symbols()) == 0

In [ ]:
DQCPass().apply(circuit)
allocator = HypergraphPartitioning()
distribution = allocator.allocate(circuit, network)
hypergraph_circuit = HypergraphCircuit(circuit)
render_circuit_jupyter(circuit)

In [ ]:
pacman_chem = PacMan(hypergraph_circuit, distribution.placement)
g_chem, topnodes_chem = pacman_chem.get_nx_graph_merged()
print(len(g_chem.nodes))
conflict_g_chem, conflict_topnodes_chem = pacman_chem.get_nx_graph_conflict()

In [ ]:
plt.figure(3,figsize=(50,45)) 
nx.draw_networkx(
    g_chem,
    pos = nx.drawing.layout.bipartite_layout(g_chem, topnodes_chem), 
    width = 1,
    node_size=1,
    font_size=20,
    with_labels=True
)

In [ ]:
nx.draw_networkx(
    conflict_g_chem,
    pos = nx.drawing.layout.bipartite_layout(conflict_g_chem, conflict_topnodes_chem), 
    width = 1)

In [ ]:
def get_packet(pacman, string_rep):
    for packet in pacman.get_all_packets():
        if packet.__repr__() == string_rep:
            return packet

print(get_packet(pacman_chem, "P88"))

## Junyi request

Adjacency matrix of the packing graph (with packets allocation)  
Adjacency matrix of intrinsic conflict graph (with packets allocation)  
Set of indecomposable hopping packets of the 6-qubit ucc circuit

In [ ]:
import sys
import numpy
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
numpy.set_printoptions(threshold=sys.maxsize, linewidth=sys.maxsize)
merged_packets = pacman_chem.get_all_merged_packets()
print(nx.to_numpy_matrix(g_chem, nodelist = merged_packets))
print(merged_packets)

In [ ]:
print(nx.to_numpy_matrix(conflict_g_chem))
print(conflict_g_chem.nodes)

In [ ]:
hopping = list()
for i in range(6):
    hopping.extend(pacman_chem.hopping_packets[i])
print(hopping)